In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, normalized_mutual_info_score
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.cluster import SpectralClustering

from sklearn.decomposition import PCA, SparsePCA, KernelPCA
from sklearn.manifold import TSNE

from rpy2.robjects import r, pandas2ri
from rpy2.robjects.vectors import StrVector

pandas2ri.activate()

import magic
import scprep

%matplotlib inline

# from sklearnex import patch_sklearn
# patch_sklearn()

import warnings

from sklearn.cluster import KMeans
from tqdm import tqdm

# r('''library(Seurat)''')

In [2]:
def get_cluster_metrics(pred, labels):
    ari_res = []
    ami_res = []
    nmi_res = []

#     try:
#         r.assign("data", pred.T)
#         seurat = r('''
#             countsData = data.frame(data)
#             pbmc <- CreateSeuratObject(counts = countsData, project = "thal_single_cell", min.cells = 1, min.features = 1)
#             pbmc <- FindVariableFeatures(pbmc, selection.method = "vst", verbose=FALSE)
#             all.genes <- rownames(pbmc)
#             pbmc <- ScaleData(pbmc, features = all.genes, verbose=FALSE)
#             pbmc <- RunPCA(pbmc, verbose=FALSE)
#             pbmc <- FindNeighbors(pbmc, verbose=FALSE)
#             pbmc <- FindClusters(pbmc, verbose=FALSE)
#             Idents(pbmc)
#         ''')
#         ari_res.append(adjusted_rand_score(labels, seurat))
#         ami_res.append(adjusted_mutual_info_score(labels, seurat))
#         nmi_res.append(normalized_mutual_info_score(labels, seurat))
#     except Exception as e:
#         pass

    pred_ = KMeans(n_clusters=len(np.unique(labels)), random_state=42).fit_predict(pred)

    ari_res.append(adjusted_rand_score(labels, pred_))
    ami_res.append(adjusted_mutual_info_score(labels, pred_))
    nmi_res.append(normalized_mutual_info_score(labels, pred_))

    warnings.filterwarnings("error")

    affinities = ['cosine', 'linear', 'poly']

    for i in affinities:
        try:
            pred_ = SpectralClustering(
                n_clusters=len(np.unique(labels)), 
                random_state=42, 
                affinity=i
            ).fit_predict(pred)
            ari_res.append(adjusted_rand_score(labels, pred_))
            ami_res.append(adjusted_mutual_info_score(labels, pred_))
            nmi_res.append(normalized_mutual_info_score(labels, pred_))
        except:
            ari_res.append(0)
            ami_res.append(0)
            nmi_res.append(0)

    warnings.resetwarnings()
    
    return max(ari_res), max(ami_res), max(nmi_res)

In [3]:
def get_data(i):
    df = pd.read_csv('../data/{}/data.csv.gz'.format(i), index_col=0)
    tmp = np.sign(df)
    cols = (np.sum(tmp) > int((df.shape[0])*0.05))
    rows = (np.sum(tmp, axis=1) > int((df.shape[1])*0.05))
    df = np.log(df.loc[rows, cols] + 1)
    df_norm = df.copy()
    df_norm = scprep.normalize.library_size_normalize(df_norm)    
    df_norm = scprep.transform.sqrt(df_norm)
    X_norm = pd.DataFrame(df_norm, columns=df.columns)
    labels = df.index
    return X_norm, labels

In [4]:
dir_list = !ls ../data/ | grep -v zip
dir_list

['baron',
 'bmcite',
 'brosens',
 'carey',
 'cbmc',
 'chang',
 'Fujii',
 'hcabm40k',
 'hrvatin',
 'jakel',
 'jiang',
 'loureiro',
 'manno',
 'mingyao',
 'pbmc3k',
 'Selewa',
 'Xu']

In [5]:
len(dir_list)

17

In [6]:
res = []
for i in tqdm(dir_list):
    X_norm, labels = get_data(i)
    pred = magic.MAGIC().fit_transform(X_norm)
    res.append(get_cluster_metrics(pred, labels))

  0%|          | 0/17 [00:00<?, ?it/s]

Calculating MAGIC...
  Running MAGIC on 1791 cells and 8669 genes.
  Calculating graph and diffusion operator...
    Calculating PCA...
    Calculated PCA in 1.07 seconds.
    Calculating KNN search...
    Calculated KNN search in 0.25 seconds.
    Calculating affinities...
    Calculated affinities in 0.24 seconds.
  Calculated graph and diffusion operator in 1.59 seconds.
  Running MAGIC with `solver='exact'` on 8669-dimensional data may take a long time. Consider denoising specific genes with `genes=<list-like>` or using `solver='approximate'`.
  Calculating imputation...
  Calculated imputation in 0.26 seconds.
Calculated MAGIC in 1.89 seconds.


  6%|▌         | 1/17 [00:15<04:01, 15.10s/it]

Calculating MAGIC...
  Running MAGIC on 5445 cells and 5421 genes.
  Calculating graph and diffusion operator...
    Calculating PCA...


/export/scratch/inoue019/envs/gpu/lib/python3.8/site-packages/magic/magic.py:650: DeprecationWarning: Call to deprecated method task. (Use TaskLogger.log_task instead) -- Deprecated since version 1.1.0.
  with _logger.task("MAGIC"):
/export/scratch/inoue019/envs/gpu/lib/python3.8/site-packages/magic/magic.py:381: DeprecationWarning: Call to deprecated method info. (Use TaskLogger.log_info instead) -- Deprecated since version 1.1.0.
  _logger.info(
/export/scratch/inoue019/envs/gpu/lib/python3.8/site-packages/magic/magic.py:436: DeprecationWarning: Call to deprecated method task. (Use TaskLogger.log_task instead) -- Deprecated since version 1.1.0.
  with _logger.task("graph and diffusion operator"):
/export/scratch/inoue019/envs/gpu/lib/python3.8/site-packages/graphtools/api.py:258: DeprecationWarning: Call to deprecated method debug. (Use TaskLogger.log_debug instead) -- Deprecated since version 1.1.0.
  _logger.debug(msg)
/export/scratch/inoue019/envs/gpu/lib/python3.8/site-packages/g

...

/export/scratch/inoue019/envs/gpu/lib/python3.8/site-packages/graphtools/graphs.py:362: DeprecationWarning: Call to deprecated method task. (Use TaskLogger.log_task instead) -- Deprecated since version 1.1.0.
  with _logger.task("affinities"):
/export/scratch/inoue019/envs/gpu/lib/python3.8/site-packages/graphtools/graphs.py:373: DeprecationWarning: Call to deprecated method debug. (Use TaskLogger.log_debug instead) -- Deprecated since version 1.1.0.
  _logger.debug(
/export/scratch/inoue019/envs/gpu/lib/python3.8/site-packages/graphtools/graphs.py:415: DeprecationWarning: Call to deprecated method debug. (Use TaskLogger.log_debug instead) -- Deprecated since version 1.1.0.
  _logger.debug(


    Calculated affinities in 1.10 seconds.
  Calculated graph and diffusion operator in 7.60 seconds.
  Running MAGIC with `solver='exact'` on 7389-dimensional data may take a long time. Consider denoising specific genes with `genes=<list-like>` or using `solver='approximate'`.
  Calculating imputation...


/export/scratch/inoue019/envs/gpu/lib/python3.8/site-packages/graphtools/base.py:559: DeprecationWarning: Call to deprecated method debug. (Use TaskLogger.log_debug instead) -- Deprecated since version 1.1.0.
  _logger.debug("Using addition symmetrization.")
/export/scratch/inoue019/envs/gpu/lib/python3.8/site-packages/magic/magic.py:586: DeprecationWarning: Call to deprecated method warning. (Use TaskLogger.log_warning instead) -- Deprecated since version 1.1.0.
  _logger.warning(
/export/scratch/inoue019/envs/gpu/lib/python3.8/site-packages/magic/magic.py:748: DeprecationWarning: Call to deprecated method task. (Use TaskLogger.log_task instead) -- Deprecated since version 1.1.0.
  with _logger.task("imputation"):


  Calculated imputation in 2.19 seconds.
Calculated MAGIC in 9.85 seconds.


100%|██████████| 17/17 [21:40<00:00, 76.52s/it]


In [11]:
pd.DataFrame(res, columns=['ARI', 'AMI', 'NMI'], index=dir_list)

,ARI,AMI,NMI
baron,0.765658,0.835604,0.839344
bmcite,0.588317,0.682699,0.683016
brosens,0.468512,0.649397,0.651711
carey,0.743721,0.778168,0.778936
cbmc,0.662201,0.746191,0.751070
chang,0.161977,0.302264,0.324759
Fujii,0.443908,0.565823,0.568675
hcabm40k,0.034687,0.051680,0.054587
hrvatin,0.777089,0.867684,0.872608
jakel,0.411675,0.587467,0.589291
